# Pandas Workflow com LlamaIndex Workflows

>Este notebook demonstra como criar um sistema de análise de dados usando LlamaIndex, Pandas e Groq para consultas em linguagem natural.


## Sobre a Atualização do LlamaIndex

O **LlamaIndex** descontinuou o módulo **QueryPipeline** em favor de uma nova abordagem chamada **Workflows**.
Essa mudança foi implementada na versão `0.11` do LlamaIndex, com o objetivo de oferecer uma arquitetura mais
flexível e escalável para a construção de aplicações de IA generativa.

### Por que o QueryPipeline foi descontinuado?
O QueryPipeline era uma API declarativa útil para orquestrar consultas simples a avançadas sobre os dados.
Porém, ele se mostrou limitado para cenários mais dinâmicos. Por isso, os **Workflows** foram introduzidos,
trazendo uma arquitetura orientada a eventos para fluxos de trabalho mais sofisticados.

### O que são Workflows?
Workflows permitem orquestrar etapas personalizadas de forma assíncrona e condicional, facilitando
integrações complexas e manipulação de dados em tempo real.

**Documentação oficial:** [docs.llamaindex.ai](https://docs.llamaindex.ai)



#Código:


##1 Instalar Dependências:

In [1]:
# Instalar dependências (-q para instalação silenciosa)
!pip install -q llama-index llama-index-llms-openai llama-index-llms-groq llama-index-experimental gradio fpdf

##1.1 Para uso no Colab


In [2]:
import nest_asyncio
nest_asyncio.apply()

##2 Imports

In [3]:
import pandas as pd
import textwrap
import re
from pydantic import BaseModel, Field, field_validator, SkipValidation
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core.workflow import Workflow, Event, StartEvent, StopEvent, step
from llama_index.core.prompts import PromptTemplate
from google.colab import userdata
import asyncio

##3 Instruções e Prompts

In [4]:
# ===== INSTRUÇÕES PARA CONVERSÃO DE CONSULTAS =====
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO FINAL.\n"
    "5. Não coloque a expressão entre aspas.\n"
    "6. Evite atribuições (=) na linha final - prefira expressões que retornem valores.\n"
    "7. Para operações de múltiplas linhas, termine com uma expressão que retorne o resultado.\n"
    "8. Exemplos válidos:\n"
    "   - df.groupby('coluna')['valor'].sum()\n"
    "   - df['coluna'].value_counts().head(5)\n"
    "   - df.describe()\n"
    "9. Evite códigos como 'df['coluna'] = valor' - prefira consultas que retornem dados.\n"
)

# ===== PROMPTS =====
pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de `print(df.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)

RESPONSE_SYNTHESIS_PROMPT_STR = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
   "Consulta: {query_str}\n\n"
   "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
   "Saída do Pandas: {pandas_output}\n\n"
   "Resposta: "
   "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}"
)

print("✅ Prompts configurados com sucesso!")

✅ Prompts configurados com sucesso!


##4 Configuração Pydantic V2

In [5]:
class LLMConfig(BaseModel):
    model: str = Field(..., description="Nome do modelo Groq a ser usado")
    api_key: str = Field(..., description="Chave da API Groq")
    data_url: str = Field(..., description="URL do CSV com os dados")

    @field_validator("data_url")
    @classmethod
    def validar_url(cls, v: str) -> str:
        if not (v.startswith("http://") or v.startswith("https://")):
            raise ValueError("data_url deve começar com http:// ou https://")
        return v

    @field_validator("api_key")
    @classmethod
    def validar_api_key(cls, v: str) -> str:
        if not v or len(v.strip()) == 0:
            raise ValueError("api_key não pode ser vazia")
        return v

print("✅ Configuração Pydantic criada!")

✅ Configuração Pydantic criada!


##5 Modelos de Eventos

In [6]:
class CodeEvent(Event):
    pandas_prompt: str
    query: str

class OutputEvent(Event):
    pandas_code: str
    pandas_output: SkipValidation
    query: str
    pandas_prompt: str

class ExecutedEvent(OutputEvent):
    pass

class ResponseEvent(StopEvent):
    query: str
    pandas_code: str
    pandas_output: SkipValidation
    resposta_final: str

print("✅ Modelos de eventos definidos!")

✅ Modelos de eventos definidos!


##6 Funções Auxiliares

In [7]:
def formatar_texto(response: str, largura: int = 100, imprimir: bool = True):
    texto_formatado = textwrap.fill(response, width=largura)
    if imprimir:
        print(texto_formatado)
    return texto_formatado

def descricao_colunas(df: pd.DataFrame) -> str:
    descricao = "\n".join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Colunas do DataFrame:\n" + descricao

def limpar_codigo_pandas(codigo: str) -> str:
    """Limpa e valida o código Pandas gerado pela LLM"""
    codigo = re.sub(r'```(?:python)?\n?', '', codigo)
    linhas = codigo.split('\n')
    codigo_limpo = []

    for linha in linhas:
        linha = linha.strip()
        if (linha and
            not linha.startswith('#') and
            not linha.startswith('//') and
            not linha.lower().startswith('resposta:') and
            not linha.lower().startswith('resultado:') and
            not linha.lower().startswith('código:')):
            codigo_limpo.append(linha)

    codigo_final = '\n'.join(codigo_limpo).strip()

    # Se há múltiplas linhas, pega a última linha como expressão final
    if '\n' in codigo_final:
        linhas_codigo = [l.strip() for l in codigo_final.split('\n') if l.strip()]
        # Pega a última linha que seja uma expressão (não atribuição)
        for linha in reversed(linhas_codigo):
            if not ('=' in linha and not linha.startswith('df[')):
                codigo_final = linha
                break

    return codigo_final

print("✅ Funções auxiliares configuradas!")

✅ Funções auxiliares configuradas!


##7 Configuração da API e Dados

In [8]:
# Configure sua chave da API Groq nos Secrets do Colab
# Vá em:  (ícone da chave) -> Add new secret
# Name: Groq_API
# Value: sua_chave_aqui

try:
    key = userdata.get("Groq_API")
    print("✅ Chave API carregada com sucesso!")
except Exception as e:
    print("❌ Erro ao carregar chave API:", e)
    print("Configure a chave 'Groq_API' nos Secrets do Colab")

# ===== CONFIGURAÇÃO E DADOS =====
config = LLMConfig(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    api_key=key,
    data_url="https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
)

df = pd.read_csv(config.data_url)
Settings.llm = Groq(model=config.model, api_key=config.api_key)

print(" Estrutura do DataFrame:")
print(df.head())
print(f"\n Colunas disponíveis:")
print(df.columns.tolist())
print(f"\n Info do DataFrame:")
print(df.info())
print(f"\n✅ Configuração completa! Dataset: {df.shape[0]} linhas, {df.shape[1]} colunas")

✅ Chave API carregada com sucesso!
 Estrutura do DataFrame:
     ID_compra filial       cidade tipo_cliente     genero       tipo_produto  \
0  750-67-8428      A  Santo André       Membro   Feminino     Saúde e Beleza   
1  226-31-3081      C  São Caetano       Normal   Feminino        Eletrônicos   
2  631-41-3108      A  Santo André       Normal  Masculino               Casa   
3  123-19-1176      A  Santo André       Membro  Masculino     Saúde e Beleza   
4  373-73-7910      A  Santo André       Normal  Masculino  Esportes e Viagem   

   preco_unitario  quantidade  imposto_5%     total        data      hora  \
0           74.69           7     26.1415  548.9715  2024-01-05  13:08:00   
1           15.28           5      3.8200   80.2200  2024-03-08  10:29:00   
2           46.33           7     16.2155  340.5255  2024-03-03  13:23:00   
3           58.22           8     23.2880  489.0480  2024-01-27  20:33:00   
4           86.31           7     30.2085  634.3785  2024-02-08  10:

##8 Classe Workflow Principal

In [9]:
class PandasWorkflow(Workflow):
    llm = Settings.llm
    workflow_steps = ["start_query", "generate_code", "execute_code", "synthesize_response"]

    @step
    async def start_query(self, ev: StartEvent) -> CodeEvent:
        colunas_info = descricao_colunas(df)
        prompt_text = pandas_prompt_str.format(
            colunas_detalhes=colunas_info,
            df_str=df.head(5),
            instruction_str=instruction_str,
            query_str=ev.query
        )
        return CodeEvent(pandas_prompt=prompt_text, query=ev.query)

    @step
    async def generate_code(self, ev: CodeEvent) -> OutputEvent:
        response = await self.llm.acomplete(ev.pandas_prompt)
        codigo_bruto = str(response).strip()
        codigo_limpo = limpar_codigo_pandas(codigo_bruto)
        print(f"🔧 Código gerado: {codigo_limpo}")
        return OutputEvent(
            pandas_code=codigo_limpo,
            pandas_output=None,
            query=ev.query,
            pandas_prompt=ev.pandas_prompt
        )

    @step
    async def execute_code(self, ev: OutputEvent) -> ExecutedEvent:
        try:
            if not ev.pandas_code or not ev.pandas_code.strip():
                resultado = "Erro: Código vazio gerado"
            else:
                codigo = ev.pandas_code.strip()

                # Validações mais flexíveis
                codigo_valido = (
                    codigo.startswith('df') or
                    codigo.startswith('(df') or
                    codigo.startswith('pd.') or
                    'df[' in codigo or
                    'df.' in codigo
                )

                if not codigo_valido:
                    resultado = f"Erro: Código deve conter referência ao DataFrame 'df', recebido: {codigo}"
                else:
                    # Executa o código com contexto completo
                    contexto = {
                        "df": df,
                        "pd": pd,
                        "__builtins__": __builtins__
                    }
                    resultado = eval(codigo, contexto)
                    print(f"✅ Resultado da execução: {resultado}")

        except Exception as e:
            resultado = f"Erro ao executar código: {str(e)}"
            print(f"❌ Erro na execução: {resultado}")

        return ExecutedEvent(
            pandas_code=ev.pandas_code,
            pandas_output=resultado,
            query=ev.query,
            pandas_prompt=ev.pandas_prompt
        )

    @step
    async def synthesize_response(self, ev: ExecutedEvent) -> ResponseEvent:
        if isinstance(ev.pandas_output, str) and ev.pandas_output.startswith("Erro"):
            resposta_final = f"Não foi possível processar a consulta: {ev.pandas_output}"
        else:
            prompt_text = RESPONSE_SYNTHESIS_PROMPT_STR.format(
                query_str=ev.query,
                pandas_instructions=ev.pandas_code,
                pandas_output=ev.pandas_output
            )
            resposta = await self.llm.acomplete(prompt_text)
            resposta_final = str(resposta).strip()
        return ResponseEvent(
            query=ev.query,
            pandas_code=ev.pandas_code,
            pandas_output=ev.pandas_output,
            resposta_final=resposta_final
        )

print("✅ Workflow PandasWorkflow criado com sucesso!")

✅ Workflow PandasWorkflow criado com sucesso!


##9 Função de Execução

In [28]:
async def executar_consulta(query: str, mostrar_detalhes: bool = True):
    """Executa uma consulta e retorna os resultados."""
    wf = PandasWorkflow(timeout=60, verbose=False)

    try:
        print(f" Processando ")
        print("-" * 50)

        result = await wf.run(query=query)

        if mostrar_detalhes:
            print(f"\n === RESULTADO FINAL ===")
            print(f"Query: {result.query}")
            print(f"Código: {result.pandas_code}")
            print(f"Resultado: {result.pandas_output}")

        print(f"\n === EXPLICAÇÃO ===")
        print("-" * 50)
        formatar_texto(result.resposta_final)

        return result

    except Exception as e:
        print(f" Erro no workflow: {str(e)}")
        return None

# Função auxiliar para múltiplas consultas
async def executar_multiplas_consultas(consultas: list):
    """Executa várias consultas em sequência."""
    resultados = []

    for i, consulta in enumerate(consultas, 1):
        print(f"\n{'='*60}")
        print(f"CONSULTA {i}/{len(consultas)}")
        print(f"{'='*60}")

        resultado = await executar_consulta(consulta)
        resultados.append(resultado)

        print("\n")  # Espaçamento entre consultas

    return resultados

print("✅ Funções de execução configuradas!")

✅ Funções de execução configuradas!


##10 Teste Rápido

In [11]:
# Teste rápido para verificar se tudo está funcionando
query_teste = "Qual é a avaliação média de cada filial?"
result = await executar_consulta(query_teste)

if result:
    print("\n Sistema funcionando perfeitamente!")
else:
    print("\n Houve algum problema - verifique a configuração da API")

 Processando: 'Qual é a avaliação média de cada filial?'
--------------------------------------------------
🔧 Código gerado: df.groupby('filial')['avaliacao'].mean()
✅ Resultado da execução: filial
A    7.027059
B    6.818072
C    7.072866
Name: avaliacao, dtype: float64

 === RESULTADO FINAL ===
Query: Qual é a avaliação média de cada filial?
Código: df.groupby('filial')['avaliacao'].mean()
Resultado: filial
A    7.027059
B    6.818072
C    7.072866
Name: avaliacao, dtype: float64

 === EXPLICAÇÃO ===
--------------------------------------------------
A avaliação média de cada filial é a seguinte:  - Filial A: 7,027059 - Filial B: 6,818072 - Filial
C: 7,072866  Esses valores indicam que a Filial C tem a melhor avaliação média, seguida de perto
pela Filial A. A Filial B tem a avaliação média mais baixa entre as três.  O código utilizado foi
df.groupby('filial')['avaliacao'].mean()

 Sistema funcionando perfeitamente!


##11 Exemplos de Uso Individual

In [12]:
# Execute uma consulta por vez para testar

# Exemplo 1
query = "Quais são os 5 produtos mais vendidos?"
result = await executar_consulta(query)

 Processando: 'Quais são os 5 produtos mais vendidos?'
--------------------------------------------------
🔧 Código gerado: df.groupby('tipo_produto')['quantidade'].sum().sort_values(ascending=False).head(5)
✅ Resultado da execução: tipo_produto
Eletrônicos            971
Alimentos e Bebidas    952
Esportes e Viagem      920
Casa                   911
Moda                   902
Name: quantidade, dtype: int64

 === RESULTADO FINAL ===
Query: Quais são os 5 produtos mais vendidos?
Código: df.groupby('tipo_produto')['quantidade'].sum().sort_values(ascending=False).head(5)
Resultado: tipo_produto
Eletrônicos            971
Alimentos e Bebidas    952
Esportes e Viagem      920
Casa                   911
Moda                   902
Name: quantidade, dtype: int64

 === EXPLICAÇÃO ===
--------------------------------------------------
Os 5 produtos mais vendidos são:  1. Eletrônicos, com 971 unidades vendidas 2. Alimentos e Bebidas,
com 952 unidades vendidas 3. Esportes e Viagem, com 920 unidade

In [13]:
# Exemplo 2
query = "Qual o total de vendas por região?"
result = await executar_consulta(query)

 Processando: 'Qual o total de vendas por região?'
--------------------------------------------------
🔧 Código gerado: df.groupby('filial')['total'].sum()
✅ Resultado da execução: filial
A    106200.3705
B    106197.6720
C    110568.7065
Name: total, dtype: float64

 === RESULTADO FINAL ===
Query: Qual o total de vendas por região?
Código: df.groupby('filial')['total'].sum()
Resultado: filial
A    106200.3705
B    106197.6720
C    110568.7065
Name: total, dtype: float64

 === EXPLICAÇÃO ===
--------------------------------------------------
Analisando os dados de vendas por região, podemos observar que a região com o maior total de vendas
foi a C1, com um total de R$ 105.687,07. Em seguida, temos a região A1, com um total de R$
102.200,37, e a região B1, com um total de R$ 101.97,20.  Aqui está o detalhamento:  - Região A1: R$
102.200,37 - Região B1: R$ 101.97,20 - Região C1: R$ 105.687,07  O código utilizado foi
df.groupby('filial')['total'].sum()


In [14]:
# Exemplo 3
query = "Qual produto tem a maior margem de lucro?"
result = await executar_consulta(query)

 Processando: 'Qual produto tem a maior margem de lucro?'
--------------------------------------------------
🔧 Código gerado: (df['total'] - (df['preco_unitario'] * df['quantidade'])).groupby(df['tipo_produto']).mean().idxmax()
✅ Resultado da execução: Casa

 === RESULTADO FINAL ===
Query: Qual produto tem a maior margem de lucro?
Código: (df['total'] - (df['preco_unitario'] * df['quantidade'])).groupby(df['tipo_produto']).mean().idxmax()
Resultado: Casa

 === EXPLICAÇÃO ===
--------------------------------------------------
O produto com a maior margem de lucro é da categoria Casa.   Isso significa que, em média, os
produtos classificados como Casa apresentam uma maior diferença entre o preço total e o custo
calculado pela multiplicação do preço unitário pela quantidade, quando comparado com outras
categorias.  O código utilizado foi: `(df['total'] - (df['preco_unitario'] *
df['quantidade'])).groupby(df['tipo_produto']).mean().idxmax()`


In [15]:
# Exemplo 4
query = "Quantas vendas foram feitas em cada mês?"
result = await executar_consulta(query)

 Processando: 'Quantas vendas foram feitas em cada mês?'
--------------------------------------------------
🔧 Código gerado: df['data'].str[:7].value_counts()
✅ Resultado da execução: data
2024-01    352
2024-03    345
2024-02    303
Name: count, dtype: int64

 === RESULTADO FINAL ===
Query: Quantas vendas foram feitas em cada mês?
Código: df['data'].str[:7].value_counts()
Resultado: data
2024-01    352
2024-03    345
2024-02    303
Name: count, dtype: int64

 === EXPLICAÇÃO ===
--------------------------------------------------
Em janeiro de 2024, foram feitas 352 vendas. Em fevereiro do mesmo ano, o número de vendas foi de
303. Já em março de 2024, foram registradas 345 vendas.  O código utilizado foi
df['data'].str[:7].value_counts()


In [24]:
query = "Qual o genêro que mais consome e o segmento do produto?"
result = await executar_consulta(query)

 Processando: 'Qual o genêro que mais consome e o segmento do produto?'
--------------------------------------------------
🔧 Código gerado: df.groupby(['genero', 'tipo_produto'])['total'].sum().reset_index().sort_values(by='total', ascending=False).head(1)
✅ Resultado da execução:      genero         tipo_produto       total
0  Feminino  Alimentos e Bebidas  33170.9175

 === RESULTADO FINAL ===
Query: Qual o genêro que mais consome e o segmento do produto?
Código: df.groupby(['genero', 'tipo_produto'])['total'].sum().reset_index().sort_values(by='total', ascending=False).head(1)
Resultado:      genero         tipo_produto       total
0  Feminino  Alimentos e Bebidas  33170.9175

 === EXPLICAÇÃO ===
--------------------------------------------------
O gênero que mais consome é o Feminino, com destaque para o segmento de Alimentos e Bebidas, que
atingiu um total de R$ 33.170,92.  O código utilizado foi df.groupby(['genero',
'tipo_produto'])['total'].sum().reset_index().sort_values(by='to

##12 Bateria de Testes

In [16]:
# Executar múltiplas consultas de uma vez
consultas_exemplo = [
    "Qual é a avaliação média de cada filial?",
    "Quais são os 5 produtos mais vendidos?",
    "Qual o total de vendas por região?",
    "Qual produto tem a maior margem de lucro?",
    "Quantas vendas foram feitas em cada mês?",
    "Qual é a receita total por categoria de produto?",
    "Qual filial teve melhor desempenho?",
    "Mostre a distribuição de preços dos produtos"
]

print(" Executando bateria de testes...")
resultados = await executar_multiplas_consultas(consultas_exemplo)

print(f"\n✅ Concluído! {len([r for r in resultados if r])} consultas processadas com sucesso.")

 Executando bateria de testes...

CONSULTA 1/8
 Processando: 'Qual é a avaliação média de cada filial?'
--------------------------------------------------
🔧 Código gerado: df.groupby('filial')['avaliacao'].mean()
✅ Resultado da execução: filial
A    7.027059
B    6.818072
C    7.072866
Name: avaliacao, dtype: float64

 === RESULTADO FINAL ===
Query: Qual é a avaliação média de cada filial?
Código: df.groupby('filial')['avaliacao'].mean()
Resultado: filial
A    7.027059
B    6.818072
C    7.072866
Name: avaliacao, dtype: float64

 === EXPLICAÇÃO ===
--------------------------------------------------
A avaliação média de cada filial é a seguinte:   - Filial A: 7,027059 - Filial B: 6,818072 - Filial
C: 7,072866  Esses valores indicam que a Filial C tem a melhor avaliação média, seguida de perto
pela Filial A. A Filial B tem a avaliação média mais baixa entre as três.  O código utilizado foi
df.groupby('filial')['avaliacao'].mean()



CONSULTA 2/8
 Processando: 'Quais são os 5 produtos mai

##13 Modo Interativo

In [33]:
# Modo interativo - digite suas próprias consultas
print(" MODO INTERATIVO")
print("Digite suas consultas (digite 'sair' para terminar)")
print("-" * 50)

while True:
    try:
        query = input("\n Sua consulta: ").strip()

        if query.lower() in ['sair', 'exit', 'quit', '']:
            print(" Até logo!")
            break

        await executar_consulta(query, mostrar_detalhes=False)

    except KeyboardInterrupt:
        print("\n Interrompido pelo usuário!")
        break
    except Exception as e:
        print(f" Erro: {e}")

 MODO INTERATIVO
Digite suas consultas (digite 'sair' para terminar)
--------------------------------------------------

 Sua consulta: Plot um gráfico baseado na melhor estratégia para crescimento a curto prazo?   Processando   Processando 
 Processando 
--------------------------------------------------
🔧 Código gerado: df.groupby('forma_pagamento')['total'].sum().sort_values(ascending=False).head(3)
✅ Resultado da execução: forma_pagamento
Dinheiro             112206.570
Carteira Digital     109993.107
Cartão de Crédito    100767.072
Name: total, dtype: float64

 === EXPLICAÇÃO ===
--------------------------------------------------
Para determinar a melhor estratégia para crescimento a curto prazo com base nos dados fornecidos,
vamos analisar os resultados da consulta. A consulta agrupou os dados por forma de pagamento e
calculou o total para cada grupo, ordenando os resultados em ordem decrescente e destacando os três
primeiros.  Os resultados mostram que:  1. **Dinheiro**: R$ 112.

##14 Análise Manual dos Dados


In [18]:
# Análise dos dados originais para comparação
print(" ANÁLISE MANUAL DOS DADOS (para comparação)")
print("="*50)

print(f"Estatísticas básicas:")
print(df.describe())

print(f"\nContagem por categoria:")
for col in df.select_dtypes(include=['object']).columns:
    print(f"\n{col}:")
    print(df[col].value_counts().head())

print(f"\nValores nulos:")
print(df.isnull().sum())

 ANÁLISE MANUAL DOS DADOS (para comparação)
Estatísticas básicas:
       preco_unitario   quantidade   imposto_5%        total   avaliacao
count     1000.000000  1000.000000  1000.000000  1000.000000  1000.00000
mean        55.672130     5.510000    15.379369   322.966749     6.97270
std         26.494628     2.923431    11.708825   245.885335     1.71858
min         10.080000     1.000000     0.508500    10.678500     4.00000
25%         32.875000     3.000000     5.924875   124.422375     5.50000
50%         55.230000     5.000000    12.088000   253.848000     7.00000
75%         77.935000     8.000000    22.445250   471.350250     8.50000
max         99.960000    10.000000    49.650000  1042.650000    10.00000

Contagem por categoria:

ID_compra:
ID_compra
849-09-3807    1
750-67-8428    1
226-31-3081    1
631-41-3108    1
123-19-1176    1
Name: count, dtype: int64

filial:
filial
A    340
B    332
C    328
Name: count, dtype: int64

cidade:
cidade
Santo André              340
São B

#Dicas e Informações

In [19]:
print("💡 DICAS PARA USO:")
print("="*50)
print("1. Para consultas complexas, seja específico")
print("2. Use nomes exatos das colunas mostradas na análise")
print("3. Experimente diferentes tipos de análise:")
print("   - Agrupamentos: 'média por categoria'")
print("   - Rankings: 'top 10 produtos'")
print("   - Filtros: 'vendas acima de X valor'")
print("   - Séries temporais: 'vendas por mês'")
print("4. O sistema mostra o código Pandas usado!")

print(f"\n INFORMAÇÕES DO DATASET:")
print(f"Suas colunas: {list(df.columns)}")
print(f"Quantidade de dados: {df.shape[0]} linhas x {df.shape[1]} colunas")
print(f"Tipos de dados: {df.dtypes.value_counts().to_dict()}")

print(f"\n SUGESTÕES DE CONSULTAS:")
example_queries = [
    "Qual a correlação entre preço e quantidade vendida?",
    "Mostre a distribuição de vendas por dia da semana",
    "Qual filial tem o melhor desempenho?",
    "Compare as vendas do primeiro e último trimestre",
    "Identifique produtos com baixo estoque",
    "Qual é o produto mais caro por categoria?",
    "Mostre as vendas médias por vendedor",
    "Qual região tem menor variação de preços?"
]

for i, query in enumerate(example_queries, 1):
    print(f"{i}. {query}")

print(f"\n MODELO UTILIZADO: {config.model}")
print(f" URL DOS DADOS: {config.data_url}")

💡 DICAS PARA USO:
1. Para consultas complexas, seja específico
2. Use nomes exatos das colunas mostradas na análise
3. Experimente diferentes tipos de análise:
   - Agrupamentos: 'média por categoria'
   - Rankings: 'top 10 produtos'
   - Filtros: 'vendas acima de X valor'
   - Séries temporais: 'vendas por mês'
4. O sistema mostra o código Pandas usado!

 INFORMAÇÕES DO DATASET:
Suas colunas: ['ID_compra', 'filial', 'cidade', 'tipo_cliente', 'genero', 'tipo_produto', 'preco_unitario', 'quantidade', 'imposto_5%', 'total', 'data', 'hora', 'forma_pagamento', 'avaliacao']
Quantidade de dados: 1000 linhas x 14 colunas
Tipos de dados: {dtype('O'): 9, dtype('float64'): 4, dtype('int64'): 1}

 SUGESTÕES DE CONSULTAS:
1. Qual a correlação entre preço e quantidade vendida?
2. Mostre a distribuição de vendas por dia da semana
3. Qual filial tem o melhor desempenho?
4. Compare as vendas do primeiro e último trimestre
5. Identifique produtos com baixo estoque
6. Qual é o produto mais caro por cate

#Notas de Uso:

*   Configuração da API: Configure sua chave Groq nos Secrets do Colab
*   Execução sequencial: Execute as células na ordem
*   Personalização: Modifique a URL dos dados na Célula 6
*   Debug: Os logs mostram o código Pandas gerado
*   Interativo: Use a Célula 12 para consultas livres

##Estrutura final:

*   Instalação automática de dependências
* Configuração completa da API
* Exploração automática dos dados
* Sistema de workflow robusto
* Exemplos prontos para uso
* Modo interativo
* Sistema de debug integrado